## I import all the required libraries for the code

In [1]:
from bs4 import BeautifulSoup

In [2]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

Fetching package metadata .............
Solving package specifications: .

# All requested packages already installed.
# packages in environment at /opt/conda/envs/DSX-Python35:
#
geopy                     1.17.0                     py_0    conda-forge
Fetching package metadata .............
Solving package specifications: .

# All requested packages already installed.
# packages in environment at /opt/conda/envs/DSX-Python35:
#
folium                    0.5.0                      py_0    conda-forge


## I pass the URL of the Wikipedia page to BeautifulSoup

In [136]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
html_file=requests.get(url)
soup=BeautifulSoup(html_file.content, 'lxml')

In [137]:
print(soup.prettify())

<!DOCTYPE html>
<html class="client-nojs" dir="ltr" lang="en">
 <head>
  <meta charset="utf-8"/>
  <title>
   List of postal codes of Canada: M - Wikipedia
  </title>
  <script>
   document.documentElement.className = document.documentElement.className.replace( /(^|\s)client-nojs(\s|$)/, "$1client-js$2" );
  </script>
  <script>
   (window.RLQ=window.RLQ||[]).push(function(){mw.config.set({"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":false,"wgNamespaceNumber":0,"wgPageName":"List_of_postal_codes_of_Canada:_M","wgTitle":"List of postal codes of Canada: M","wgCurRevisionId":867606113,"wgRevisionId":867606113,"wgArticleId":539066,"wgIsArticle":true,"wgIsRedirect":false,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Communications in Ontario","Postal codes in Canada","Toronto","Ontario-related lists"],"wgBreakFrames":false,"wgPageContentLanguage":"en","wgPageContentModel":"wikitext","wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wg

## This extracts the title of the webpage

In [138]:
print(soup.title.text)

List of postal codes of Canada: M - Wikipedia


### Since we are interested in the data in the table, this next step extracts the required data from the webpage

In [139]:
table=soup.find('table', class_='wikitable')

### I extract the table header from the "th" tag

In [140]:
#This extracts the columns, and passes them into a list
columns=[]

for i in table.find_all('th'):
    columns.append(i.text)
    
columns=[w.replace("\n",'') for w in columns]

columns

['Postcode', 'Borough', 'Neighbourhood']

### I extract the row data from the "td" tags

In [141]:
#This extracts the row data, and passes them into a list
rows=[]

for i in table.find_all('td'):
    rows.append(i.text)

rows=[w.replace("\n",'') for w in rows]

rows

['M1A',
 'Not assigned',
 'Not assigned',
 'M2A',
 'Not assigned',
 'Not assigned',
 'M3A',
 'North York',
 'Parkwoods',
 'M4A',
 'North York',
 'Victoria Village',
 'M5A',
 'Downtown Toronto',
 'Harbourfront',
 'M5A',
 'Downtown Toronto',
 'Regent Park',
 'M6A',
 'North York',
 'Lawrence Heights',
 'M6A',
 'North York',
 'Lawrence Manor',
 'M7A',
 "Queen's Park",
 'Not assigned',
 'M8A',
 'Not assigned',
 'Not assigned',
 'M9A',
 'Etobicoke',
 'Islington Avenue',
 'M1B',
 'Scarborough',
 'Rouge',
 'M1B',
 'Scarborough',
 'Malvern',
 'M2B',
 'Not assigned',
 'Not assigned',
 'M3B',
 'North York',
 'Don Mills North',
 'M4B',
 'East York',
 'Woodbine Gardens',
 'M4B',
 'East York',
 'Parkview Hill',
 'M5B',
 'Downtown Toronto',
 'Ryerson',
 'M5B',
 'Downtown Toronto',
 'Garden District',
 'M6B',
 'North York',
 'Glencairn',
 'M7B',
 'Not assigned',
 'Not assigned',
 'M8B',
 'Not assigned',
 'Not assigned',
 'M9B',
 'Etobicoke',
 'Cloverdale',
 'M9B',
 'Etobicoke',
 'Islington',
 'M9B',
 

In [142]:
#I reshape the row data to match the desired df
a=int(len(rows)/3)
x=np.reshape(rows, (a,3))

In [143]:
toronto_df=pd.DataFrame()
toronto_df=toronto_df.from_records(x, columns=columns)

In [144]:
toronto_df.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [145]:
toronto_df.shape

(289, 3)

In [146]:
toronto_df['Borough'].replace('Not assigned', np.NaN, inplace=True)
toronto_df.head()

,Postcode,Borough,Neighbourhood
0,M1A,NaN,Not assigned
1,M2A,NaN,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [147]:
toronto_df.dropna(subset=['Borough'], inplace=True)

In [148]:
toronto_df.head()

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights


In [149]:
toronto_df['Neighbourhood'].replace('Not assigned',toronto_df['Borough'], inplace=True)

In [150]:
toronto_df.head()

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights


In [151]:
foo=lambda a: "/".join(a)
toronto_df=toronto_df.groupby(['Postcode','Borough']).agg({'Neighbourhood': foo})

In [152]:
toronto_df.reset_index(inplace=True)

In [153]:
toronto_df

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,Rouge/Malvern
1,M1C,Scarborough,Highland Creek/Rouge Hill/Port Union
2,M1E,Scarborough,Guildwood/Morningside/West Hill
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,East Birchmount Park/Ionview/Kennedy Park
7,M1L,Scarborough,Clairlea/Golden Mile/Oakridge
8,M1M,Scarborough,Cliffcrest/Cliffside/Scarborough Village West
9,M1N,Scarborough,Birch Cliff/Cliffside West


In [154]:
toronto_df.shape

(103, 3)

In [155]:
toronto_df

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,Rouge/Malvern
1,M1C,Scarborough,Highland Creek/Rouge Hill/Port Union
2,M1E,Scarborough,Guildwood/Morningside/West Hill
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,East Birchmount Park/Ionview/Kennedy Park
7,M1L,Scarborough,Clairlea/Golden Mile/Oakridge
8,M1M,Scarborough,Cliffcrest/Cliffside/Scarborough Village West
9,M1N,Scarborough,Birch Cliff/Cliffside West


## I extract the content of the file, and modify the string to an array

In [156]:
url = 'https://cocl.us/Geospatial_data'
html_file=requests.get(url).content

In [157]:
lonlat=[]
lonlat=html_file.decode('ascii').replace("\r\n",',').split(',')

In [158]:
len(lonlat)

312

In [159]:
a=int(len(lonlat)/3)
x=np.reshape(lonlat, (a,3))

In [160]:
columns=x[0]
rows=x[1:]

In [161]:
lonlat_df=pd.DataFrame
lonlat_df=lonlat_df.from_records(rows, columns=columns)

In [162]:
lonlat_df.rename(columns={'Postal Code':'Postcode'},inplace=True)

In [163]:
lonlat_df.set_index('Postcode',inplace=True)

In [164]:
lonlat_df.head()

,Latitude,Longitude
Postcode,,
M1B,43.8066863,-79.1943534
M1C,43.7845351,-79.1604971
M1E,43.7635726,-79.1887115
M1G,43.7709921,-79.2169174
M1H,43.773136,-79.2394761


In [165]:
toronto_df.set_index('Postcode',inplace=True)

In [166]:
toronto_df=toronto_df.join(lonlat_df,how='inner').reset_index()

In [167]:
toronto_df

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,Rouge/Malvern,43.8066863,-79.1943534
1,M1C,Scarborough,Highland Creek/Rouge Hill/Port Union,43.7845351,-79.1604971
2,M1E,Scarborough,Guildwood/Morningside/West Hill,43.7635726,-79.1887115
3,M1G,Scarborough,Woburn,43.7709921,-79.2169174
4,M1H,Scarborough,Cedarbrae,43.773136,-79.2394761
5,M1J,Scarborough,Scarborough Village,43.7447342,-79.2394761
6,M1K,Scarborough,East Birchmount Park/Ionview/Kennedy Park,43.7279292,-79.2620294
7,M1L,Scarborough,Clairlea/Golden Mile/Oakridge,43.7111117,-79.2845772
8,M1M,Scarborough,Cliffcrest/Cliffside/Scarborough Village West,43.716316,-79.2394761
9,M1N,Scarborough,Birch Cliff/Cliffside West,43.692657,-79.2648481


## I get the coordinates of Toronto, CA

In [168]:
address = 'Toronto, Canada'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = float(location.latitude)
longitude = float(location.longitude)
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

/opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/geopy/geocoders/osm.py:143: UserWarning: Using Nominatim with the default "geopy/1.17.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  UserWarning


The geograpical coordinate of Toronto are 43.653963, -79.387207.


## I display the map of Toronto, to display the neighbourhoods in it

In [169]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(toronto_df['Latitude'], toronto_df['Longitude'], toronto_df['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(location=[float(lat),float(lng)],radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

## I choose to analyze and cluster the data of North York

In [170]:
scar_df=toronto_df[toronto_df['Borough']=='North York'].reset_index(drop=True)

In [171]:
scar_df.rename(columns={'Neighbourhood':'Neighborhood'},inplace=True)

Let's get the geographical coordinates of North York

In [198]:
address='North York, Toronto'
geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Manhattan are {}, {}.'.format(latitude, longitude))

/opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/geopy/geocoders/osm.py:143: UserWarning: Using Nominatim with the default "geopy/1.17.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  UserWarning


The geograpical coordinate of Manhattan are 43.7709163, -79.4124102.


Let's visualize North York, and the neighborhoods in it

In [199]:
map_scar = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(scar_df['Latitude'], scar_df['Longitude'], scar_df['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(location=[float(lat),float(lng)],radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_scar)  
    
map_scar

Now we use the Foursquare API to explore Scarborough

## Define Foursquare credentials and Version

In [200]:
CLIENT_ID = 'OQ4BQT5A4EMQXUKLVKCRFMDMCDCLN35HK1WW0IPLPZBJATL2' # your Foursquare ID
CLIENT_SECRET = 'LO5D3FTRIEXWEXPOAPR0DBXOM3SW3A43VW2XD24YINPV1QHG' # your Foursquare Secret
VERSION = '20181124' # Foursquare API version
LIMIT=100

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: OQ4BQT5A4EMQXUKLVKCRFMDMCDCLN35HK1WW0IPLPZBJATL2
CLIENT_SECRET:LO5D3FTRIEXWEXPOAPR0DBXOM3SW3A43VW2XD24YINPV1QHG


## Let's create a function to extract 100 venues in Scarborough

In [201]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [202]:
scar_venues=getNearbyVenues(scar_df['Neighborhood'],scar_df['Latitude'],scar_df['Longitude'])

Hillcrest Village
Fairview/Henry Farm/Oriole
Bayview Village
Silver Hills/York Mills
Newtonbrook/Willowdale
Willowdale South
York Mills West
Willowdale West
Parkwoods
Don Mills North
Flemingdon Park/Don Mills South
Bathurst Manor/Downsview North/Wilson Heights
Northwood Park/York University
CFB Toronto/Downsview East
Downsview West
Downsview Central
Downsview Northwest
Victoria Village
Bedford Park/Lawrence Manor East
Lawrence Heights/Lawrence Manor
Glencairn
Maple Leaf Park/North Park/Upwood Park
Humber Summit
Emery/Humberlea


Let's check how many venues were found for each neighbourhood

In [203]:
scar_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Bathurst Manor/Downsview North/Wilson Heights,17,17,17,17,17,17
Bayview Village,4,4,4,4,4,4
Bedford Park/Lawrence Manor East,24,24,24,24,24,24
CFB Toronto/Downsview East,3,3,3,3,3,3
Don Mills North,6,6,6,6,6,6
Downsview Central,3,3,3,3,3,3
Downsview Northwest,5,5,5,5,5,5
Downsview West,4,4,4,4,4,4
Emery/Humberlea,1,1,1,1,1,1


Let's find out how many unique values of venues were returned

In [204]:
print('There are {} uniques categories.'.format(len(scar_venues['Venue Category'].unique())))

There are 107 uniques categories.


## Let's analyze each neighbourhood

In [205]:
# one hot encoding
scar_onehot = pd.get_dummies(scar_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
scar_onehot['Neighborhood']=scar_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [scar_onehot.columns[-1]] + list(scar_onehot.columns[:-1])
scar_onehot = scar_onehot[fixed_columns]

scar_onehot.head()

,Neighborhood,Accessories Store,Airport,American Restaurant,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Bakery,Bank,Bar,Baseball Field,Basketball Court,Beer Store,Bike Shop,Boutique,Bridal Shop,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Station,Bus Stop,Butcher,Cafeteria,Café,Candy Store,Caribbean Restaurant,Chinese Restaurant,Clothing Store,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping,Cosmetics Shop,Deli / Bodega,Department Store,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Electronics Store,Empanada Restaurant,Event Space,Fast Food Restaurant,Food & Drink Shop,Food Court,Food Truck,Fraternity House,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,General Entertainment,Gift Shop,Golf Course,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hardware Store,Hockey Arena,Hotel,Ice Cream Shop,Indian Restaurant,Indonesian Restaurant,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Korean Restaurant,Liquor Store,Lounge,Luggage Store,Massage Studio,Mediterranean Restaurant,Metro Station,Middle Eastern Restaurant,Miscellaneous Shop,Movie Theater,Music Store,Nail Salon,Park,Pet Store,Pharmacy,Pizza Place,Plaza,Pool,Portuguese Restaurant,Pub,Ramen Restaurant,Record Shop,Restaurant,Salon / Barbershop,Sandwich Place,Shoe Store,Shopping Mall,Smoothie Shop,Spa,Sporting Goods Shop,Steakhouse,Sushi Restaurant,Tailor Shop,Tea Room,Thai Restaurant,Theater,Toy / Game Store,Video Game Store,Video Store,Vietnamese Restaurant,Wings Joint,Women's Store
0,Hillcrest Village,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Hillcrest Village,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Hillcrest Village,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Hillcrest Village,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Fairview/Henry Farm/Oriole,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [206]:
scar_onehot.shape

(234, 108)

## Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [207]:
scar_grouped = scar_onehot.groupby('Neighborhood').mean().reset_index()
scar_grouped.head()

,Neighborhood,Accessories Store,Airport,American Restaurant,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Bakery,Bank,Bar,Baseball Field,Basketball Court,Beer Store,Bike Shop,Boutique,Bridal Shop,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Station,Bus Stop,Butcher,Cafeteria,Café,Candy Store,Caribbean Restaurant,Chinese Restaurant,Clothing Store,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping,Cosmetics Shop,Deli / Bodega,Department Store,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Electronics Store,Empanada Restaurant,Event Space,Fast Food Restaurant,Food & Drink Shop,Food Court,Food Truck,Fraternity House,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,General Entertainment,Gift Shop,Golf Course,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hardware Store,Hockey Arena,Hotel,Ice Cream Shop,Indian Restaurant,Indonesian Restaurant,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Korean Restaurant,Liquor Store,Lounge,Luggage Store,Massage Studio,Mediterranean Restaurant,Metro Station,Middle Eastern Restaurant,Miscellaneous Shop,Movie Theater,Music Store,Nail Salon,Park,Pet Store,Pharmacy,Pizza Place,Plaza,Pool,Portuguese Restaurant,Pub,Ramen Restaurant,Record Shop,Restaurant,Salon / Barbershop,Sandwich Place,Shoe Store,Shopping Mall,Smoothie Shop,Spa,Sporting Goods Shop,Steakhouse,Sushi Restaurant,Tailor Shop,Tea Room,Thai Restaurant,Theater,Toy / Game Store,Video Game Store,Video Store,Vietnamese Restaurant,Wings Joint,Women's Store
0,Bathurst Manor/Downsview North/Wilson Heights,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.058824,0.0,0.000000,0.0,0.0,0.0,0.0,0.058824,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.00,0.0,0.117647,0.000000,0.0,0.0,0.058824,0.0,0.0,0.058824,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.058824,0.058824,0.0,0.0,0.058824,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.058824,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.058824,0.058824,0.0,0.000000,0.0,0.000000,0.0,0.0,0.058824,0.0,0.058824,0.0,0.058824,0.0,0.0,0.0,0.0,0.058824,0.0,0.0,0.000000,0.0,0.000000,0.0,0.058824,0.0,0.0,0.0
1,Bayview Village,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.250000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.250000,0.0,0.000000,0.25,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.250000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0
2,Bedford Park/Lawrence Manor East,0.0,0.000000,0.041667,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.041667,0.0,0.041667,0.0,0.000000,0.00,0.0,0.083333,0.041667,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.083333,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.041667,0.041667,0.0,0.000000,0.041667,0.0,0.0,0.000000,0.041667,0.0,0.083333,0.000000,0.0,0.041667,0.0,0.041667,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.041667,0.041667,0.0,0.000000,0.0,0.041667,0.0,0.0,0.041667,0.0,0.041667,0.0,0.000000,0.0,0.0,0.0,0.0,0.041667,0.0,0.0,0.041667,0.0,0.041667,0.0,0.000000,0.0,0.0,0.0
3,CFB Toronto/Downsview East,0.0,0.333333,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.333333,0.000000,0.0,0.000000,0.0,0.000000,0.00,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.333333,0.0,0.000000,0.000000

## Let's sort the neighbourhood using the top 5 venues

First, let's write a function to sort the venues in descending order.

In [208]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now let's create a new dataframe to display the top 5 venues for each neighborhood.

In [209]:
num_top_venues = 5

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = scar_grouped['Neighborhood']

for ind in np.arange(scar_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(scar_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,Bathurst Manor/Downsview North/Wilson Heights,Coffee Shop,Ice Cream Shop,Deli / Bodega,Pharmacy,Pizza Place
1,Bayview Village,Chinese Restaurant,Café,Bank,Japanese Restaurant,Women's Store
2,Bedford Park/Lawrence Manor East,Italian Restaurant,Coffee Shop,Fast Food Restaurant,Comfort Food Restaurant,Hardware Store
3,CFB Toronto/Downsview East,Park,Airport,Bus Stop,Empanada Restaurant,Comfort Food Restaurant
4,Don Mills North,Gym / Fitness Center,Caribbean Restaurant,Pool,Café,Baseball Field
5,Downsview Central,Food Truck,Baseball Field,Korean Restaurant,Women's Store,Event Space
6,Downsview Northwest,Grocery Store,Gym / Fitness Center,Athletics & Sports,Discount Store,Liquor Store
7,Downsview West,Grocery Store,Shopping Mall,Bank,Comfort Food Restaurant,Construction & Landscaping
8,Emery/Humberlea,Baseball Field,Women's Store,Event Space,Comfort Food Restaurant,Construction & Landscaping
9,Fairview/Henry Farm/Oriole,Clothing Store,Fast Food Restaurant,Coffee Shop,Women's Store,Asian Restaurant


## Here I cluster the neighbourhoods
Run k-means to cluster the neighborhood into 5 clusters.

In [210]:
# set number of clusters
kclusters = 5

scar_grouped_clustering = scar_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(scar_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_

array([0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 4, 0, 0, 0,
       3], dtype=int32)

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [211]:
scar_merged = scar_df

# add clustering labels
scar_merged['Cluster Labels'] = kmeans.labels_

scar_merged = scar_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

scar_merged.head() # check the last columns!

,Postcode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,M2H,North York,Hillcrest Village,43.8037622,-79.3634517,0,Golf Course,Pool,Mediterranean Restaurant,Dog Run,Electronics Store
1,M2J,North York,Fairview/Henry Farm/Oriole,43.7785175,-79.3465557,0,Clothing Store,Fast Food Restaurant,Coffee Shop,Women's Store,Asian Restaurant
2,M2K,North York,Bayview Village,43.7869473,-79.385975,0,Chinese Restaurant,Café,Bank,Japanese Restaurant,Women's Store
3,M2L,North York,Silver Hills/York Mills,43.7574902,-79.3747141,0,Cafeteria,Women's Store,Empanada Restaurant,Comfort Food Restaurant,Construction & Landscaping
4,M2M,North York,Newtonbrook/Willowdale,43.789053,-79.4084928,0,Gym,Clothing Store,Comfort Food Restaurant,Construction & Landscaping,Cosmetics Shop


Finally, let's visualize the resulting clusters

In [212]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(scar_merged['Latitude'], scar_merged['Longitude'], scar_merged['Neighborhood'], scar_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [float(lat),float(lon)],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters